# Import packages

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import Prob_models as PM
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import tensorboard
tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions
import random
from netCDF4 import Dataset


# Set random seed for reproducibility

In [2]:
seed = 100
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Training radius model

### Radius dataset

##### Synthetic dataset

In [12]:
A = 2*tfd.Uniform().sample(1000)
X = tfd.InverseGamma(concentration =1.5, scale = 0.6 ).sample(1000)
R1 = A*X


random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
train_dataset = R1[:250]
train_dataset = tf.reshape(train_dataset,[250,1])
eval_dataset = R1[250:]
eval_dataset=tf.reshape(eval_dataset,[750,1])
print(np.max(R1))
print(np.max(R1[:250]))

165.50166
16.883898


##### Danube river network

In [3]:
ncfile = Dataset('/home/nlafon/These/4Dvarnetstochastic/Danube_river_network/Dataset_danube.nc',"r")
L=[]
for i in range(31):
    L.append(ncfile['S'+str(i+1)][:].reshape(18244,1))
        
dataset = np.concatenate((L[0],L[1],L[2],L[3],L[4],L[5],L[6],L[7],L[8],L[9],L[10],L[11],L[12],L[13],L[14],L[15],L[16],L[17],L[18],L[19],L[20],L[21],L[22],L[23],L[24],L[25],L[26],L[27],L[28],L[29],L[30]),axis=1)
print(dataset.shape)


(18244, 31)


We focus on two groups of stations R4 and R2 (see $Asadi\ et\ al.\ (2015). $)  
R2 comprises five stations in the Inn basin that are fed by precipitation in high-altitude alpine regions.  
R4 contains five stations with sources north of the Danube.  
Stations of R4 : 23; 24; 25; 26; 27.   
Stations of R2 : 13; 28; 29; 30; 31.

In [4]:
R4 = dataset[:,22:27]
axis = 1
R4_rad = np.sum(R4,axis)
R4_rad = R4_rad.reshape((18244,1))
R4_S = np.divide(R4,R4_rad)
train_dataset = R4_rad[::25,:]
eval_dataset  = R4_rad
print(train_dataset.shape)
print(np.max(train_dataset))
print(np.max(R4_rad))


(730, 1)
856.9
1899.0


### Import VAE model

Importing standard VAE model : vae = PM.Std_VAE()  
Importing Extreme value VAE model assuming known shape parameter: vae = PM.Ext_VAE()  
Importing Extreme value VAE model, shape parameter is learned : vae = PM.U_Ext_VAE()  

In [22]:
#vae = PM.Std_VAE()
#vae = PM.Ext_VAE()
vae  = PM.U_Ext_VAE()

<tf.Variable 'prior_c:0' shape=(1,) dtype=float32, numpy=array([4.8579035], dtype=float32)>


### Training procedure

In [38]:
checkpoint_filepath = '/home/nlafon/These/Extreme_VAE/tmp/Danube/radius/Ext_VAE/Gamma_output_prior_unknown3/checkpoint'
metric ='val_loss'
model_checkpoint_callback = tfk.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor=metric,
    mode='min',
    save_best_only=True)

##### Pre-training of the encoder if needed in case of instability

In [16]:
# U_Ext_VAE
enc = vae.encoder
enc.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-5),
            loss=negative_log_likelihood)

enc.fit(x=train_dataset,
	y=train_dataset, 
        #validation_data=(eval_dataset,eval_dataset), 
        batch_size=32,
        epochs=10)
print(enc.prior.concentration)

Epoch 1/10
23/23 [==============================] - 1s 5ms/step - loss: 586.1639
Epoch 2/10
23/23 [==============================] - 0s 4ms/step - loss: 567.6381
Epoch 3/10
23/23 [==============================] - 0s 4ms/step - loss: 543.5135
Epoch 4/10
23/23 [==============================] - 0s 4ms/step - loss: 510.9303
Epoch 5/10
23/23 [==============================] - 0s 4ms/step - loss: nan 
Epoch 6/10
23/23 [==============================] - 0s 4ms/step - loss: nan
Epoch 7/10
23/23 [==============================] - 0s 4ms/step - loss: nan
Epoch 8/10
23/23 [==============================] - 0s 4ms/step - loss: nan
Epoch 9/10
23/23 [==============================] - 0s 4ms/step - loss: nan
Epoch 10/10
23/23 [==============================] - 0s 4ms/step - loss: nan
<tf.Variable 'prior_c:0' shape=(1,) dtype=float32, numpy=array([nan], dtype=float32)>


In [39]:
negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)

vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
            loss=negative_log_likelihood)

##### Fit the VAE, saving the best model wrt to val loss

In [40]:
vae.fit(train_dataset,train_dataset, 
        validation_data=(eval_dataset,eval_dataset), 
        batch_size=32,
        epochs=5000, 
        callbacks = [model_checkpoint_callback]
       )

Epoch 1/1000
23/23 [==============================] - 6s 214ms/step - loss: 7.5493 - val_loss: 7.1321
Epoch 2/1000
23/23 [==============================] - 3s 136ms/step - loss: 6.5737 - val_loss: 6.4667
Epoch 3/1000
23/23 [==============================] - 3s 123ms/step - loss: 6.1812 - val_loss: 6.7813
Epoch 4/1000
23/23 [==============================] - 2s 84ms/step - loss: 6.2231 - val_loss: 6.2562
Epoch 5/1000
23/23 [==============================] - 2s 80ms/step - loss: 6.2738 - val_loss: 6.5242
Epoch 6/1000
23/23 [==============================] - 2s 86ms/step - loss: 6.2533 - val_loss: 6.4655
Epoch 7/1000
23/23 [==============================] - 2s 80ms/step - loss: 6.2852 - val_loss: 6.4543
Epoch 8/1000
23/23 [==============================] - 3s 121ms/step - loss: 6.2199 - val_loss: 6.1478
Epoch 9/1000
23/23 [==============================] - 3s 141ms/step - loss: 6.0658 - val_loss: 6.2226
Epoch 10/1000
23/23 [==============================] - 2s 93ms/step - loss: 6.0393 - v

KeyboardInterrupt: 

In [34]:
enc = vae.encoder
print(enc.prior.concentration)

<tf.Variable 'prior_c:0' shape=(1,) dtype=float32, numpy=array([4.4771185], dtype=float32)>


# Training radius-conditioned angular model

### Angle dataset

##### Synthetic dataset

Let **X** the multivariate random vector we want to sample from. **X** =R **$\Theta$**  
The norm is set to the 1D samples of previous section. 
We sample on the multivariate sphere according to a Dirichlet distribution. 
For every radius r,  **$\Theta$** | r is Dirichlet with K=5, $\alpha_1(R))=\alpha_2(R) = 3\left(2 -min(1, \frac{1}{2R})\right)$, $ \alpha_3=\alpha_4 = \alpha_5=3\left(1 + min(1, \frac{1}{2||R||})\right)$.   


In [18]:
def alphas_function(x):
    res = tf.convert_to_tensor([6. -3*tf.minimum(1.,0.5/x), 6. -3*tf.minimum(1.,0.5/x) , 3.+ 3*tf.minimum(1,0.5/x), 3.+ 3*tf.minimum(1,0.5/x), 3.+ 3*tf.minimum(1,0.5/x)])
    return(tf.transpose(res))


alphas          = alphas_function(train_dataset[:,0])
angle_train_dep = tfd.Dirichlet(alphas).sample()
print(angle_train_dep.shape)
print(train_dataset.shape)
alphas_eval     = alphas_function(eval_dataset[:,0])
angle_eval_dep = tfd.Dirichlet(alphas_eval).sample()

(250, 5)
(250, 1)


##### Danube dataset

In [5]:
angle_train_dep = R4_S[::25,:]
print(angle_train_dep.shape)
print(train_dataset.shape)
angle_eval_dep  = R4_S
print(angle_eval_dep.shape)

(730, 5)
(730, 1)
(18244, 5)


## Model training 

In [8]:
checkpoint_filepath = '/home/nlafon/These/Extreme_VAE/tmp/Danube/angle/normal_output2/checkpoint'
metric ='val_loss'
model_checkpoint_callback = tfk.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor=metric,
    mode='min',
    save_best_only=True)

In [7]:
vae = PM.Sphere_VAE()
#If training from previously learned model :
#vae.load_weights(checkpoint_filepath)

2023-01-11 11:08:52.370692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 11:08:52.373747: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
vae.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-5),
            loss=negative_log_likelihood)

In [10]:
vae.fit([angle_train_dep,train_dataset],angle_train_dep,
        validation_data=([angle_eval_dep,eval_dataset],angle_eval_dep),
        batch_size=32,epochs=5000,
       callbacks = [model_checkpoint_callback])

Epoch 1/5000
23/23 [==============================] - 4s 114ms/step - loss: -7.8900 - val_loss: -7.7346
Epoch 2/5000
23/23 [==============================] - 2s 95ms/step - loss: -7.9944 - val_loss: -7.6563
Epoch 3/5000
23/23 [==============================] - 2s 82ms/step - loss: -7.9769 - val_loss: -7.6996
Epoch 4/5000
23/23 [==============================] - 2s 78ms/step - loss: -7.9934 - val_loss: -7.6917
Epoch 5/5000
23/23 [==============================] - 3s 125ms/step - loss: -7.9413 - val_loss: -7.7356
Epoch 6/5000
23/23 [==============================] - 3s 125ms/step - loss: -7.8925 - val_loss: -7.7178
Epoch 7/5000
23/23 [==============================] - 2s 90ms/step - loss: -7.8636 - val_loss: -7.7445
Epoch 8/5000
23/23 [==============================] - 2s 84ms/step - loss: -7.9511 - val_loss: -7.7303
Epoch 9/5000
23/23 [==============================] - 3s 122ms/step - loss: -7.9822 - val_loss: -7.7134
Epoch 10/5000
23/23 [==============================] - 2s 90ms/step -